# Santander 2025.2 (Explorando IA Generativa em um Pipeline de ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SD2025.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sd-2025-prd.up.railway.app/users/{id}` (API da Santander 2025) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sd-2025-prd.up.railway.app/users/{id}`.



In [16]:
sd2025_api_url = "https://desafio-santander-2025-ia-generativa-pipeline-production.up.railway.app/"

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [17]:
import pandas as pd

df = pd.read_csv('SD2025.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5]


In [18]:
import requests
import json


import requests
import json

def get_user(id):
  response = requests.get(f'{sd2025_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "name": "Ana Souza",
    "email": "ana@example.com",
    "balance": 5000.0,
    "news": []
  },
  {
    "id": 2,
    "name": "Carlos Lima",
    "email": "carlos@example.com",
    "balance": 12000.0,
    "news": []
  },
  {
    "id": 3,
    "name": "Marina Alves",
    "email": "marina@example.com",
    "balance": 8000.0,
    "news": []
  },
  {
    "id": 4,
    "name": "Pedro Santos",
    "email": "pedro@example.com",
    "balance": 1000.0,
    "news": []
  },
  {
    "id": 5,
    "name": "Rafaela Silva",
    "email": "rafaela@example.com",
    "balance": 3000.0,
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [9]:
pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 3.4 MB/s eta 0:00:00


In [10]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
groq_api_key = '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'

In [13]:
from groq import Groq

client = Groq(api_key=groq_api_key)

def generate_ai_news(user):
    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "Você é um expert em aplicações financeiras."},
            {"role": "user", "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"}
        ]
    )

    # AQUI: trocar ["content"] por .content
    return completion.choices[0].message.content.strip('"')

for user in users:
    news = generate_ai_news(user)
    print(news)
    user["news"].append({"description": news})




Rafaella, invista em seu futuro! Divida seus objetivos financeiros e comece a construir uma reserva líquida agora mesmo!
Carlos, investimentos ajudam a construir seu futuro financeiro, aumentar riqueza e alcançar seus objetivos!


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [19]:
import requests

def update_user(user):
    response = requests.put(f"{sd2025_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}")


User Ana Souza updated? True
User Carlos Lima updated? True
User Marina Alves updated? True
User Pedro Santos updated? True
User Rafaela Silva updated? True
